<a href="https://colab.research.google.com/github/mlabonne/linear-programming-course/blob/main/2_Integer_vs_Linear_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integer vs. Linear Programming

> Chapter 2 of the [Linear Programming Course](https://github.com/mlabonne/Linear-Programming-Course)

❤️ Created by [@maximelabonne](https://twitter.com/maximelabonne).

Companion notebook to execute the code from the following article: https://towardsdatascience.com/integer-programming-vs-linear-programming-in-python-f1be5bb4e60e

In [1]:
!python -m pip install --upgrade --user -q ortools

# MIP solution with CBC

The goal is to maximize the army with the following resources: 1200 food, 800 wood, 600 gold.

<br/>

| Unit | 🌾Food | 🪵Wood | 🪙Gold | 💪Power |
| :--- | :---: | :---: | :---: | :---: |
| 🗡️Swordsman | 60 | 20 | 0 | 70 |
| 🏹Bowman | 80 | 10 | 40 | 95 |
| 🐎Horseman | 140 | 0 |100 | 230 |

<br/>

If the following cell fails, click on `Runtime > Restart and run all`.

In [2]:
# Import OR-Tools wrapper for linear programming
from ortools.linear_solver import pywraplp

# Create the linear solver using the CBC backend
solver = pywraplp.Solver('Maximize army power', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# 1. Create the variables we want to optimize
swordsmen = solver.IntVar(0, solver.infinity(), 'swordsmen')
bowmen = solver.IntVar(0, solver.infinity(), 'bowmen')
horsemen = solver.IntVar(0, solver.infinity(), 'horsemen')

# 2. Add constraints for each resource
solver.Add(swordsmen*60 + bowmen*80 + horsemen*140 <= 1200)
solver.Add(swordsmen*20 + bowmen*10 <= 800)
solver.Add(bowmen*40 + horsemen*100 <= 600)

# 3. Maximize the objective function
solver.Maximize(swordsmen*70 + bowmen*95 + horsemen*230)

# Solve problem
status = solver.Solve()

# If an optimal solution has been found, print results
if status == pywraplp.Solver.OPTIMAL:
  print('================= Solution =================')
  print(f'Solved in {solver.wall_time():.2f} milliseconds in {solver.iterations()} iterations')
  print()
  print(f'Optimal value = {solver.Objective().Value()} 💪power')
  print('Army:')
  print(f' - 🗡️Swordsmen = {swordsmen.solution_value()}')
  print(f' - 🏹Bowmen = {bowmen.solution_value()}')
  print(f' - 🐎Horsemen = {horsemen.solution_value()}')
else:
  print('The solver could not find an optimal solution.')

================= Solution =================
Solved in 4.00 milliseconds in 0 iterations

Optimal value = 1800.0 💪power
Army:
 - 🗡️Swordsmen = 6.0
 - 🏹Bowmen = 0.0
 - 🐎Horsemen = 6.0


# Abstract model as a Python function

Same problem but with a function and parameters instead of a static model.

In [3]:
# Inputs
UNITS = ['🗡️Swordsmen', '🏹Bowmen', '🐎Horsemen']

DATA = [[60, 20, 0, 70],
        [80, 10, 40, 95],
        [140, 0, 100, 230]]

RESOURCES = [1200, 800, 600]


def solve_army(UNITS, DATA, RESOURCES):
    # Create the linear solver using the CBC backend
    solver = pywraplp.Solver('Maximize army power', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    # 1. Create the variables we want to optimize
    units = [solver.IntVar(0, solver.infinity(), unit) for unit in UNITS]

    # 2. Add constraints for each resource
    for r, _ in enumerate(RESOURCES):
      solver.Add(sum(DATA[u][r] * units[u] for u, _ in enumerate(units)) <= RESOURCES[r])

    # 3. Maximize the objective function
    solver.Maximize(sum(DATA[u][-1] * units[u] for u, _ in enumerate(units)))

    # Solve problem
    status = solver.Solve()

    # If an optimal solution has been found, print results
    if status == pywraplp.Solver.OPTIMAL:
      print('================= Solution =================')
      print(f'Solved in {solver.wall_time():.2f} milliseconds in {solver.iterations()} iterations')
      print()
      print(f'Optimal value = {solver.Objective().Value()} 💪power')
      print('Army:')
      for u, _ in enumerate(units):
        print(f' - {units[u].name()} = {units[u].solution_value()}')
    else:
      print('The solver could not find an optimal solution.')

solve_army(UNITS, DATA, RESOURCES)

================= Solution =================
Solved in 3.00 milliseconds in 0 iterations

Optimal value = 1800.0 💪power
Army:
 - 🗡️Swordsmen = 6.0
 - 🏹Bowmen = 0.0
 - 🐎Horsemen = 6.0


# Optimize a new problem

The goal is to maximize the power of the army with the following resources: 183000 food, 90512 wood, 80150 gold.

<br/>

| Unit | 🌾Food | 🪵Wood | 🪙Gold | 💪Attack | ❤️Health
| :--- | :---: | :---: | :---: | :---: | :---: |
| 🗡️Swordsman      | 60  | 20  | 0   | 6   | 70 |
| 🛡️Man-at-arms    | 100 | 0   | 20  | 12  | 155 |
| 🏹Bowman         | 30  | 50  | 0   | 5   | 70 |
| ❌Crossbowman    | 80  | 0   | 40  | 12  | 80 |
| 🔫Handcannoneer  | 120 | 0   | 120 | 35  | 150 |
| 🐎Horseman       | 100 | 20  | 0   | 9   | 125 |
| ♞Knight          | 140 | 0   | 100 | 24  | 230 | 
| 🐏Battering ram  | 0   | 300 | 0   | 200 | 700 |
| 🎯Springald      | 0   | 250 | 250 | 30 | 200 |

</br>

In [4]:
UNITS = [
    '🗡️Swordsmen',
    '🛡️Men-at-arms',
    '🏹Bowmen',
    '❌Crossbowmen',
    '🔫Handcannoneers',
    '🐎Horsemen',
    '♞Knights',
    '🐏Battering rams',
    '🎯Springalds',
    '🪨Mangonels',
]

DATA = [
    [60, 20, 0, 6, 70],
    [100, 0, 20, 12, 155],
    [30, 50, 0, 5, 70],
    [80, 0, 40, 12, 80],
    [120, 0, 120, 35, 150],
    [100, 20, 0, 9, 125],
    [140, 0, 100, 24, 230],
    [0, 300, 0, 200, 700],
    [0, 250, 250, 30, 200],
    [0, 400, 200, 12*3, 240]
]

RESOURCES = [183000, 90512, 80150]


def solve_army(UNITS, DATA, RESOURCES):
  # Create the linear solver using the CBC backend
  solver = pywraplp.Solver('Maximize army power', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

  # 1. Create the variables we want to optimize
  units = [solver.IntVar(0, solver.infinity(), unit) for unit in UNITS]

  # 2. Add constraints for each resource
  for r, _ in enumerate(RESOURCES):
    solver.Add(sum(DATA[u][r] * units[u] for u, _ in enumerate(units)) <= RESOURCES[r])

  # 3. Maximize the new objective function
  solver.Maximize(sum((10*DATA[u][-2] + DATA[u][-1]) * units[u] for u, _ in enumerate(units)))

  # Solve problem
  status = solver.Solve()

  # If an optimal solution has been found, print results
  if status == pywraplp.Solver.OPTIMAL:
    print('================= Solution =================')
    print(f'Solved in {solver.wall_time():.2f} milliseconds in {solver.iterations()} iterations')
    print()
    print(f'Optimal value = {solver.Objective().Value()} 💪power')
    print('Army:')
    for u, _ in enumerate(units):
      print(f' - {units[u].name()} = {units[u].solution_value()}')
  else:
    print('The solver could not find an optimal solution.')

solve_army(UNITS, DATA, RESOURCES)

================= Solution =================
Solved in 113.00 milliseconds in 412 iterations

Optimal value = 1393145.0 💪power
Army:
 - 🗡️Swordsmen = 2.0
 - 🛡️Men-at-arms = 1283.0
 - 🏹Bowmen = 3.0
 - ❌Crossbowmen = 0.0
 - 🔫Handcannoneers = 454.0
 - 🐎Horsemen = 0.0
 - ♞Knights = 0.0
 - 🐏Battering rams = 301.0
 - 🎯Springalds = 0.0
 - 🪨Mangonels = 0.0


# Minimize resource consumption

The goal is to minimize the resources that are used to get an army power >= 1,000,001.

In [10]:
def solve_army(UNITS, DATA, RESOURCES):
  # Create the linear solver using the CBC backend
  solver = pywraplp.Solver('Minimize resource consumption', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

  # 1. Create the variables we want to optimize
  units = [solver.IntVar(0, solver.infinity(), unit) for unit in UNITS]

  # 2. Add power constraint
  solver.Add(sum((10 * DATA[u][-2] + DATA[u][-1]) * units[u] for u, _ in enumerate(units)) >= 1000001)

  # 3. Minimize the objective function
  solver.Minimize(sum((DATA[u][0] + DATA[u][1] + DATA[u][2]) * units[u] for u, _ in enumerate(units)))

  # Solve problem
  status = solver.Solve()

  # If an optimal solution has been found, print results
  if status == pywraplp.Solver.OPTIMAL:
    print('================= Solution =================')
    print(f'Solved in {solver.wall_time():.2f} milliseconds in {solver.iterations()} iterations')
    print()

    power = sum((10 * DATA[u][-2] + DATA[u][-1]) * units[u].solution_value() for u, _ in enumerate(units))
    print(f'Optimal value = {solver.Objective().Value()} 🌾🪵🪙resources')
    print(f'Power = 💪{power}')
    print('Army:')
    for u, _ in enumerate(units):
      print(f' - {units[u].name()} = {units[u].solution_value()}')
    print()

    food = sum((DATA[u][0]) * units[u].solution_value() for u, _ in enumerate(units))
    wood = sum((DATA[u][1]) * units[u].solution_value() for u, _ in enumerate(units))
    gold = sum((DATA[u][2]) * units[u].solution_value() for u, _ in enumerate(units))
    print('Resources:')
    print(f' - 🌾Food = {food}')
    print(f' - 🪵Wood = {wood}')
    print(f' - 🪙Gold = {gold}')
  else:
    print('The solver could not find an optimal solution.')

solve_army(UNITS, DATA, RESOURCES)

================= Solution =================
Solved in 3.00 milliseconds in 0 iterations

Optimal value = 111300.0 🌾🪵🪙resources
Power = 💪1001700.0
Army:
 - 🗡️Swordsmen = 0.0
 - 🛡️Men-at-arms = 0.0
 - 🏹Bowmen = 0.0
 - ❌Crossbowmen = 0.0
 - 🔫Handcannoneers = 0.0
 - 🐎Horsemen = 0.0
 - ♞Knights = 0.0
 - 🐏Battering rams = 371.0
 - 🎯Springalds = 0.0
 - 🪨Mangonels = 0.0

Resources:
 - 🌾Food = 0.0
 - 🪵Wood = 111300.0
 - 🪙Gold = 0.0


# Minimize resource consumption + limited resources

The goal is to minimize the resources that are used to get an army power > 1,000,00 but the resources cannot exceed: 183000 food, 90512 wood, 80150 gold.

In [6]:
def solve_army(UNITS, DATA, RESOURCES):
  # Create the linear solver using the CBC backend
  solver = pywraplp.Solver('Minimize resource consumption', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

  # 1. Create the variables we want to optimize
  units = [solver.IntVar(0, solver.infinity(), unit) for unit in UNITS]

  # 2. Add constraints for each resource
  for r, _ in enumerate(RESOURCES):
    solver.Add(sum((10 * DATA[u][-2] + DATA[u][-1]) * units[u] for u, _ in enumerate(units)) >= 1000001)

  # Old constraints for limited resources
  for r, _ in enumerate(RESOURCES):
    solver.Add(sum(DATA[u][r] * units[u] for u, _ in enumerate(units)) <= RESOURCES[r])

  # 3. Minimize the objective function
  solver.Minimize(sum((DATA[u][0] + DATA[u][1] + DATA[u][2]) * units[u] for u, _ in enumerate(units)))

  # Solve problem
  status = solver.Solve()

  # If an optimal solution has been found, print results
  if status == pywraplp.Solver.OPTIMAL:
    print('================= Solution =================')
    print(f'Solved in {solver.wall_time():.2f} milliseconds in {solver.iterations()} iterations')
    print()

    power = sum((10 * DATA[u][-2] + DATA[u][-1]) * units[u].solution_value() for u, _ in enumerate(units))
    print(f'Optimal value = {solver.Objective().Value()} 🌾🪵🪙resources')
    print(f'Power = 💪{power}')
    print('Army:')
    for u, _ in enumerate(units):
      print(f' - {units[u].name()} = {units[u].solution_value()}')
    print()
    
    food = sum((DATA[u][0]) * units[u].solution_value() for u, _ in enumerate(units))
    wood = sum((DATA[u][1]) * units[u].solution_value() for u, _ in enumerate(units))
    gold = sum((DATA[u][2]) * units[u].solution_value() for u, _ in enumerate(units))
    print('Resources:')
    print(f' - 🌾Food = {food}')
    print(f' - 🪵Wood = {wood}')
    print(f' - 🪙Gold = {gold}')
  else:
    print('The solver could not find an optimal solution.')

solve_army(UNITS, DATA, RESOURCES)

================= Solution =================
Solved in 31.00 milliseconds in 1 iterations

Optimal value = 172100.0 🌾🪵🪙resources
Power = 💪1000105.0
Army:
 - 🗡️Swordsmen = 1.0
 - 🛡️Men-at-arms = 681.0
 - 🏹Bowmen = 0.0
 - ❌Crossbowmen = 0.0
 - 🔫Handcannoneers = 0.0
 - 🐎Horsemen = 0.0
 - ♞Knights = 0.0
 - 🐏Battering rams = 301.0
 - 🎯Springalds = 0.0
 - 🪨Mangonels = 0.0

Resources:
 - 🌾Food = 68160.0
 - 🪵Wood = 90320.0
 - 🪙Gold = 13620.0
